In [2]:
# Data Collection

!pip install kaggle --upgrade
!pip3 install Pillow

import kaggle
import os
import json
import random
from datetime import datetime
from PIL import Image, ImageStat
from PIL.ExifTags import TAGS
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)

def get_predominant_colors(image_file, numcolors=3, resize=150, output = []):
    # Resize image to speed up processing
    img = Image.open(image_file)
    img = img.copy()
    img.thumbnail((resize, resize))
 
    # Reduce to palette
    paletted = img.convert('P', palette=Image.ADAPTIVE, colors=numcolors)
 
    # Find dominant colors
    palette = paletted.getpalette()
    color_counts = sorted(paletted.getcolors(), reverse=True)
    colors = list()
    for i in range(numcolors):
        palette_index = color_counts[i][1]
        dominant_color = palette[palette_index*3:palette_index*3+3]
        colors.append(tuple(dominant_color))
    return colors

def convert_rgb_to_names(rgb_tuple):
    
    # a dictionary of all the hex and their respective names in css3
    css3_db = CSS3_HEX_TO_NAMES
    names = []
    rgb_values = []    
    for color_hex, color_name in css3_db.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))
    
    kdt_db = KDTree(rgb_values)    
    distance, index = kdt_db.query(rgb_tuple)
    return int(index)

In [9]:
## You have to collect and download a set of images. You have the following tasks to program, automating the process as much as possible:
### 1. Create a folder called images.

!mkdir -p ./images
!mkdir -p ./metadata

### 2. Download open-licensed images to the folder images (minimum 100 images).

kaggle.api.authenticate()
kaggle.api.dataset_download_files('nielspace/pexels-mountain-images', path='./images', unzip=True)    
    
### 3. Save metadata of every image like image size, image format (.jpeg, .png, etc.), image orientation (landscape, portrait, square, etc.), creation date, camera model, etc. in one or more JSON files. You can make use of the Exif information present in the image files.

directory = './images/Mountain'
metadata = {}
orientation = ""
images_data =[]
for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        image_data = {}
        path, file_extension = os.path.splitext(f)
        time_creation = os.path.getmtime(f)
        im = Image.open(f)
        size = im.size
        colors = {}
        predominant_colors = get_predominant_colors(directory + "/" + filename)
        colors["predominant_colors"] = []
        for predominant_color in predominant_colors:
            colors["predominant_colors"].append(convert_rgb_to_names(predominant_color))
        for idx,val in enumerate(["red","green","blue"]):
            colors[val] = Image.Image.getextrema(im)[idx]
            colors[val] = colors[val] + (ImageStat.Stat(im).median[idx],)
        print(colors)
        if size[0]>size[1] :
            orientation = "landscape"
        elif size[0] == size[1] :
            orientation = "square"
        else :
            orientation = "portrait"
        metadata[filename] = {
            "file_extension": file_extension,
            "creation_date": datetime.fromtimestamp(time_creation/1000.0).strftime("%m/%d/%Y, %H:%M:%S"),
            "size": {
                "width":size[0],
                "height":size[1]
            },
            "orientation": orientation,
            "colors": {
                "predominant_colors":{
                    "first_color":colors["predominant_colors"][0],
                    "second_color":colors["predominant_colors"][1],
                    "third_color":colors["predominant_colors"][2]
                },
                "red": {
                    "minimum":colors["red"][0],
                    "maximum":colors["red"][1],
                    "median":colors["red"][2]
                },
                "green": {
                    "minimum":colors["green"][0],
                    "maximum":colors["green"][1],
                    "median":colors["green"][2]
                },
                "blue": {
                    "minimum":colors["blue"][0],
                    "maximum":colors["blue"][1],
                    "median":colors["blue"][2]
                }
            }
        }
        print(metadata[filename])


with open("./metadata/metadata.json", "w") as outfile:
    json.dump(metadata, outfile, indent=4)

        

{'predominant_colors': [27, 25, 12], 'red': (0, 255, 169), 'green': (0, 255, 141), 'blue': (0, 255, 72)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 3500, 'height': 2333}, 'orientation': 'landscape', 'colors': {'predominant_colors': {'first_color': 27, 'second_color': 25, 'third_color': 12}, 'red': {'minimum': 0, 'maximum': 255, 'median': 169}, 'green': {'minimum': 0, 'maximum': 255, 'median': 141}, 'blue': {'minimum': 0, 'maximum': 255, 'median': 72}}}
{'predominant_colors': [34, 72, 120], 'red': (0, 255, 115), 'green': (0, 255, 122), 'blue': (0, 255, 129)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 3264, 'height': 4759}, 'orientation': 'portrait', 'colors': {'predominant_colors': {'first_color': 34, 'second_color': 72, 'third_color': 120}, 'red': {'minimum': 0, 'maximum': 255, 'median': 115}, 'green': {'minimum': 0, 'maximum': 255, 'median': 122}, 'blue': {'minimum': 0, 'maximum': 255, 'median': 129}

{'predominant_colors': [34, 34, 72], 'red': (0, 255, 13), 'green': (1, 245, 85), 'blue': (0, 240, 107)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 4912, 'height': 7360}, 'orientation': 'portrait', 'colors': {'predominant_colors': {'first_color': 34, 'second_color': 34, 'third_color': 72}, 'red': {'minimum': 0, 'maximum': 255, 'median': 13}, 'green': {'minimum': 1, 'maximum': 245, 'median': 85}, 'blue': {'minimum': 0, 'maximum': 240, 'median': 107}}}
{'predominant_colors': [34, 39, 49], 'red': (0, 255, 88), 'green': (0, 255, 85), 'blue': (0, 255, 96)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 6914, 'height': 4463}, 'orientation': 'landscape', 'colors': {'predominant_colors': {'first_color': 34, 'second_color': 39, 'third_color': 49}, 'red': {'minimum': 0, 'maximum': 255, 'median': 88}, 'green': {'minimum': 0, 'maximum': 255, 'median': 85}, 'blue': {'minimum': 0, 'maximum': 255, 'median': 96}}}
{'predo

{'predominant_colors': [62, 73, 72], 'red': (0, 255, 166), 'green': (0, 254, 193), 'blue': (6, 255, 220)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 6000, 'height': 4000}, 'orientation': 'landscape', 'colors': {'predominant_colors': {'first_color': 62, 'second_color': 73, 'third_color': 72}, 'red': {'minimum': 0, 'maximum': 255, 'median': 166}, 'green': {'minimum': 0, 'maximum': 254, 'median': 193}, 'blue': {'minimum': 6, 'maximum': 255, 'median': 220}}}
{'predominant_colors': [124, 134, 23], 'red': (10, 255, 252), 'green': (11, 255, 252), 'blue': (6, 255, 252)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 4160, 'height': 5200}, 'orientation': 'portrait', 'colors': {'predominant_colors': {'first_color': 124, 'second_color': 134, 'third_color': 23}, 'red': {'minimum': 10, 'maximum': 255, 'median': 252}, 'green': {'minimum': 11, 'maximum': 255, 'median': 252}, 'blue': {'minimum': 6, 'maximum': 255, 'media

{'predominant_colors': [34, 126, 23], 'red': (0, 255, 64), 'green': (30, 248, 127), 'blue': (1, 255, 164)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 4288, 'height': 2848}, 'orientation': 'landscape', 'colors': {'predominant_colors': {'first_color': 34, 'second_color': 126, 'third_color': 23}, 'red': {'minimum': 0, 'maximum': 255, 'median': 64}, 'green': {'minimum': 30, 'maximum': 248, 'median': 127}, 'blue': {'minimum': 1, 'maximum': 255, 'median': 164}}}
{'predominant_colors': [78, 27, 119], 'red': (0, 255, 93), 'green': (0, 255, 52), 'blue': (0, 255, 43)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 6000, 'height': 3376}, 'orientation': 'landscape', 'colors': {'predominant_colors': {'first_color': 78, 'second_color': 27, 'third_color': 119}, 'red': {'minimum': 0, 'maximum': 255, 'median': 93}, 'green': {'minimum': 0, 'maximum': 255, 'median': 52}, 'blue': {'minimum': 0, 'maximum': 255, 'median': 43}}

{'predominant_colors': [120, 72, 45], 'red': (0, 255, 174), 'green': (0, 255, 181), 'blue': (0, 255, 202)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 5440, 'height': 8160}, 'orientation': 'portrait', 'colors': {'predominant_colors': {'first_color': 120, 'second_color': 72, 'third_color': 45}, 'red': {'minimum': 0, 'maximum': 255, 'median': 174}, 'green': {'minimum': 0, 'maximum': 255, 'median': 181}, 'blue': {'minimum': 0, 'maximum': 255, 'median': 202}}}
{'predominant_colors': [39, 66, 45], 'red': (0, 255, 204), 'green': (0, 255, 204), 'blue': (0, 255, 204)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 5472, 'height': 2976}, 'orientation': 'landscape', 'colors': {'predominant_colors': {'first_color': 39, 'second_color': 66, 'third_color': 45}, 'red': {'minimum': 0, 'maximum': 255, 'median': 204}, 'green': {'minimum': 0, 'maximum': 255, 'median': 204}, 'blue': {'minimum': 0, 'maximum': 255, 'median': 20

{'predominant_colors': [34, 77, 112], 'red': (22, 255, 97), 'green': (19, 255, 81), 'blue': (0, 255, 71)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 3648, 'height': 5472}, 'orientation': 'portrait', 'colors': {'predominant_colors': {'first_color': 34, 'second_color': 77, 'third_color': 112}, 'red': {'minimum': 22, 'maximum': 255, 'median': 97}, 'green': {'minimum': 19, 'maximum': 255, 'median': 81}, 'blue': {'minimum': 0, 'maximum': 255, 'median': 71}}}
{'predominant_colors': [34, 34, 72], 'red': (0, 255, 31), 'green': (0, 255, 50), 'blue': (0, 255, 74)}
{'file_extension': '.jpg', 'creation_date': '01/20/1970, 02:13:58', 'size': {'width': 6000, 'height': 4000}, 'orientation': 'landscape', 'colors': {'predominant_colors': {'first_color': 34, 'second_color': 34, 'third_color': 72}, 'red': {'minimum': 0, 'maximum': 255, 'median': 31}, 'green': {'minimum': 0, 'maximum': 255, 'median': 50}, 'blue': {'minimum': 0, 'maximum': 255, 'median': 74}}}
{'p

In [10]:
# Labeling and Annotation

directory = './images/Mountain'
data_path = "./metadata/metadata.json"


if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)
        
for filename in os.listdir(directory) :
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        tags = {}
        tags["like"] = random.randint(0,100)
        tags["hashtag"] = "moutain"
        json_data[filename]["tags"] = tags
        
with open("./metadata/metadata.json", "w") as outfile:
    json.dump(json_data, outfile, indent=4)

In [11]:
from sklearn import tree
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Data Analyses

numb_users = 1
directory = './images/Mountain'
data_path = "./metadata/metadata.json"
rand = 0

# Metadata for each file
if os.path.isfile(data_path):
    with open(data_path) as target:
        json_data = json.load(target)

# Creating data for each user
json_data_users = {}        
for i in range (0, numb_users):
    images_per_users = []
    tags_per_users = []
    rand = random.randint(1, 4)
    if rand == 1:
        tags_per_users.append(["like", "colors"])
    elif rand == 2:
        tags_per_users.append("colors")
    elif rand == 3:
        tags_per_users.append(["hashtag", "colors"])
    else:
        tags_per_users.append(["like, hashtag", "colors"])
    for filename in os.listdir(directory) :
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            rand = random.randint(1, 3)
            if rand == 1 :
                images_per_users.append(filename)
                print(filename)
    json_data_users[i] = {
        "images": images_per_users,
        "tags": tags_per_users
    }


pexels-lisa-fotios-1036371.jpg
pexels-eberhard-grossgasteiger-1366909.jpg
pexels-markus-spiske-143577.jpg
pexels-eberhard-grossgasteiger-443446.jpg
pexels-eberhard-grossgasteiger-1699030.jpg
pexels-lukas-kloeppel-577696.jpg
pexels-eberhard-grossgasteiger-1366901.jpg
pexels-pixabay-266691.jpg
pexels-aleksey-kuprikov-3551227.jpg
pexels-stephan-seeber-1054201.jpg
pexels-aron-visuals-1643113.jpg
pexels-pixabay-417265.jpg
pexels-vittorio-staffolani-655674.jpg
pexels-archie-binamira-913215.jpg
pexels-denis-linine-714258.jpg
pexels-sebastian-palomino-1853371.jpg
pexels-quang-nguyen-vinh-2166695.jpg
pexels-eberhard-grossgasteiger-976917.jpg
pexels-zhaocan-li-1755243.jpg
pexels-xue-guangjian-1731427.jpg
pexels-may-barros-1260841.jpg
pexels-stephan-seeber-1130847.jpg
pexels-johannes-rapprich-1482937.jpg
pexels-pixabay-52710.jpg
pexels-snapwire-37728.jpg
pexels-lukas-kloeppel-545964.jpg
pexels-eberhard-grossgasteiger-1743401.jpg
pexels-eberhard-grossgasteiger-1287145.jpg
pexels-eberhard-grossgast

In [12]:
# Creating dataframes to predict what the user might like
data = []
result = []
json_data_learning = dict(list(json_data.items())[len(json_data)//3:])
for i in json_data_learning:
    if i in json_data_users[0]["images"]:
        result.append('Favorite')
    else:
        result.append('NotFavorite')
    data.append([json_data_learning[i]["colors"]["predominant_colors"]["first_color"],
                 json_data_learning[i]["colors"]["predominant_colors"]["second_color"],
                 json_data_learning[i]["colors"]["predominant_colors"]["third_color"],
                 json_data_learning[i]["colors"]["red"]["maximum"],
                 json_data_learning[i]["colors"]["red"]["median"],
                 json_data_learning[i]["colors"]["green"]["maximum"],
                 json_data_learning[i]["colors"]["green"]["median"],
                 json_data_learning[i]["colors"]["blue"]["maximum"],
                 json_data_learning[i]["colors"]["blue"]["median"],
                 json_data_learning[i]["orientation"], 
                 json_data_learning[i]["size"]["width"],
                 json_data_learning[i]["size"]["height"]
                ])

dataframe = pd.DataFrame(data, columns=['first_color', 'second_color', 'third_color','max_red', 'median_red','max_green', 'median_green','max_blue', 'median_blue','orientation', 'width', 'height'])
resultframe = pd.DataFrame(result, columns=['favorite'])
dataframe

,first_color,second_color,third_color,max_red,median_red,max_green,median_green,max_blue,median_blue,orientation,width,height
0,62,73,72,255,166,254,193,255,220,landscape,6000,4000
1,124,134,23,255,252,255,252,255,252,portrait,4160,5200
2,34,120,112,255,125,255,116,255,109,portrait,3648,5472
3,72,33,23,255,103,251,123,255,150,landscape,7782,5191
4,34,66,45,242,191,243,192,245,194,portrait,3648,5472
...,...,...,...,...,...,...,...,...,...,...,...,...
63,7,23,27,255,52,255,29,255,8,landscape,4935,3293
64,23,120,45,255,205,255,183,254,188,landscape,3648,2432
65,34,49,127,255,102,255,122,255,115,landscape,7952,5304
66,34,66,39,255,55,250,69,255,96,portrait,2048,2560


In [ ]:
import os
import math 
import matplotlib.pyplot as plot
isLiked = resultframe["favorite"]=="Favorite"
dataframeLiked = dataframe.filter(items = resultframe[isLiked]["favorite"].index, axis = 0)
grouped = dataframeLiked.groupby(['orientation']).count()
dataframeLiked
grouped = grouped.rename(columns={'width':'count'})["count"].reset_index()
grouped.plot(x=0, kind='bar', title="Orientations liked")

In [ ]:
#generating numerical labels
le1 = LabelEncoder()
dataframe['orientation'] = le1.fit_transform(dataframe['orientation'])

le2 = LabelEncoder()
resultframe['favorite'] = le2.fit_transform(resultframe['favorite'])

dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(dataframe, resultframe)

prediction = dtc.predict([
        [1,
         2,
         3,
         le1.transform(['landscape'])[0],
         12,
         23
        ]])
print(le2.inverse_transform(prediction))
print(dtc.feature_importances_)